## Predicting future sales.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector as msql
from mysql.connector import Error
import scipy.stats as st

import data_loader_module
import get_load_SQL


### Import data with SQL

In [3]:
columns = ("date", "date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day", "shop_name", "item_name", "item_category_id", "item_category_name")

In [4]:
user_data = {"user":"root", "password":"3629"}
data_base = "ayn_sales_db"
table_name = "sales_train_mod"
query_lang = "SELECT * FROM ayn_sales_db.sales_train_mod"

#data_frames_dict = {"sales_df":sql_query(user_data=user_data, data_base=data_base, table_name=table_name, query_lang=query_lang)}

### Import data from .csv

In [5]:
# Load the CSV info into a dataframe.
path = "C:\\Users\\Alejandro\\Desktop\\Alpha Project\\Code\\Supermarket_challenge\\Data_future_sales\\Translated DF\\Sales_train_mod.csv"
df_dict = {}
df_dict["sales_df"] = pd.read_csv(path, index_col=[0])

In [6]:
# Convert Data Type

# to Category
df_dict["sales_df"][["shop_id", "item_id", "shop_name", "item_name", "item_category_id","item_category_name"]] = df_dict["sales_df"][["shop_id", "item_id", "shop_name", "item_name", "item_category_id","item_category_name"]].astype("category")

# to Date Time
df_dict["sales_df"]["date"] = pd.to_datetime(df_dict["sales_df"]["date"], infer_datetime_format=True, dayfirst=True)


In [7]:
df_dict["sales_df_raw"] = df_dict["sales_df"].copy(deep=True)

### Exploratory Data Analysis

The objective is to gain more knowledge about their operations.

In this regard the main purpose is to be able to predict the futures sales. But also to get some insights regarding they daily operations from 2013 to octover 2015.

Muchas  


In [8]:
df_dict["sales_df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   date_block_num      int64         
 2   shop_id             category      
 3   item_id             category      
 4   item_price          float64       
 5   item_cnt_day        float64       
 6   shop_name           category      
 7   item_name           category      
 8   item_category_id    category      
 9   item_category_name  category      
dtypes: category(6), datetime64[ns](1), float64(2), int64(1)
memory usage: 135.7 MB


In [9]:
df_dict["sales_df"]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.0,1.0,"Yaroslavl shopping center ""Altair""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray
1,2013-01-02,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray
2,2013-01-03,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray
3,2013-01-20,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray
4,2013-01-23,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray
...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-22,33,55,13093,250.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 250 rubles [Digital vers...,36,Payment Cards - Windows (Digital)
2935845,2015-09-21,32,55,13091,1000.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 1000 rubles [Digital ver...,36,Payment Cards - Windows (Digital)
2935846,2015-09-16,32,55,13094,2500.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 2500 rubles [Digital ver...,36,Payment Cards - Windows (Digital)
2935847,2015-09-22,32,55,13094,2500.0,2.0,Digital warehouse 1C-Online,Windows Payment Card: 2500 rubles [Digital ver...,36,Payment Cards - Windows (Digital)


In [10]:
df_dict["sales_df"].describe()

,date_block_num,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,8.908532e+02,1.242641e+00
std,9.422988e+00,1.729800e+03,2.618834e+00
min,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.990000e+02,1.000000e+00
75%,2.300000e+01,9.990000e+02,1.000000e+00
max,3.300000e+01,3.079800e+05,2.169000e+03


#### **Item category**
**About this data:**

*Knowldege*
* There are 84 distinct items categories in the raw data. <span style="color:orange">*| This data doesn't represents the true categories of each product in a hierichal point of view, using a split for dividing the data in 3 categories: market_category "21" → item_category → item*</span>


In [11]:
# We create a list with every single one of the item category distintct values
list(df_dict["sales_df"]["item_category_name"].unique())

['Cinema - Blu-Ray',
 'Cinema - DVD',
 'Accessories - PSVita',
 'Accessories - PS3',
 'PC Games - Standard Editions',
 'Games - PS3',
 'Music - locally produced CD',
 'Music - Music Video',
 'Games - PSVita',
 'Games - XBOX 360',
 'Games - PSP',
 'Gifts - Gadgets, robots, sports',
 'PC Games - Extra Editions',
 'Payment cards - PSN',
 'Programs - For home and office',
 "PC Games - Collector's Editions",
 'Gifts - Soft toys',
 'Cinema - Blu-Ray 3D',
 'Accessories - XBOX 360',
 'Game consoles - XBOX 360',
 'Payment cards - Live!',
 'Gifts - Souvenirs',
 'Games - Accessories for games',
 'Game consoles - PSP',
 'Accessories - PSP',
 'Music - branded CD',
 'Books - Methodological materials 1C',
 'Cinema - Collectible',
 'Music - MP3',
 'Gifts - Board games',
 'Gifts - Board games (compact)',
 'Gifts - Development',
 'Gifts - Figurines',
 'Batteries',
 'Accessories - PS4',
 'Gifts - Souvenirs (in bulk)',
 'Game consoles - PS3',
 'Music - Gift editions',
 'Programs - 1C: Enterprise 8',
 'Gam

In [12]:
# As we could see there are repetitive items on each category so we could create another one.
df_dict["sales_df"][["market_category", "item_category"]] = df_dict["sales_df"]["item_category_name"].str.split(" - ", expand=True, )
df_dict["sales_df"]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,market_category,item_category
0,2013-01-02,0,59,22154,999.0,1.0,"Yaroslavl shopping center ""Altair""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray
1,2013-01-02,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray
2,2013-01-03,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray
3,2013-01-20,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray
4,2013-01-23,0,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",PHENOMENON 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray
...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-22,33,55,13093,250.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 250 rubles [Digital vers...,36,Payment Cards - Windows (Digital),Payment Cards,Windows (Digital)
2935845,2015-09-21,32,55,13091,1000.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 1000 rubles [Digital ver...,36,Payment Cards - Windows (Digital),Payment Cards,Windows (Digital)
2935846,2015-09-16,32,55,13094,2500.0,1.0,Digital warehouse 1C-Online,Windows Payment Card: 2500 rubles [Digital ver...,36,Payment Cards - Windows (Digital),Payment Cards,Windows (Digital)
2935847,2015-09-22,32,55,13094,2500.0,2.0,Digital warehouse 1C-Online,Windows Payment Card: 2500 rubles [Digital ver...,36,Payment Cards - Windows (Digital),Payment Cards,Windows (Digital)


In [13]:
#
print("There are: ",len(list(df_dict["sales_df"]["market_category"].unique())) , "Distinct Market Categories in the stores")
print("There are: ",len(list(df_dict["sales_df"]["item_category_name"].unique())) , " Distinct item categories in the stores")
print("There are: ",len(list(df_dict["sales_df"]["item_name"].unique())), "Distinct Items in the stores")


There are:  21 Distinct Market Categories in the stores
There are:  84  Distinct item categories in the stores
There are:  21784 Distinct Items in the stores


In [14]:
df_dict["items_cat_df"] = df_dict["sales_df"].groupby("market_category")["item_name"].unique().to_frame()

In [15]:
df_dict["items_cat_df"]["count_of_items"] = [len(list(df_dict["items_cat_df"]["item_name"][row])) for row in range(df_dict["items_cat_df"]["item_name"].count())]
df_dict["items_cat_df"] = df_dict["items_cat_df"].sort_values(by="count_of_items", ascending=False)
df_dict["items_cat_df"]["perc__distinct_items"] = ((df_dict["items_cat_df"]["count_of_items"]/df_dict["items_cat_df"]["count_of_items"].sum())*100).round(2)
df_dict["items_cat_df"]


,item_name,count_of_items,perc__distinct_items
market_category,,,
Cinema,"['PHENOMENON 2012 (BD)', 'AVATAR (BD+DVD)', 'A...",7375,33.85
Music,"['RAMAZZOTTI EROS Noi', 'PRODIGY The Fat Of Th...",4256,19.54
Gifts,['Powerball 250Hz NEON Pro Amber (PB - 188LC A...,3460,15.88
PC Games,"['RAGE [PC, Jewel, Russian version]', 'Rayman ...",2127,9.76
Games,"['RAGE Anarchy Edition [PS3 Russian version]',...",1732,7.95
Books,"['Hello, 1C. An example of rapid application d...",1228,5.64
Programs,"['Movavi Video Editor (box)', 'MOVAVI Video Su...",1134,5.21
Accessories,['PS Vita: 4gamers Travel Case Blue (SPC9001BL...,217,1.00
Game consoles,['Xbox 360 250 GB + Kinect bundle matte+Dance ...,144,0.66


In [16]:
df_dict["items_df"] = df_dict["sales_df"].groupby(["market_category", "item_category"])["item_name"].unique().to_frame()

In [17]:
df_dict["items_df"]["count_of_items"] = [len(list(df_dict["items_df"]["item_name"][row])) for row in range(df_dict["items_df"]["item_name"].count())]
df_dict["items_df"] = df_dict["items_df"].sort_values(by="count_of_items", ascending=False)
df_dict["items_df"]["perc_distinct_items"] = ((df_dict["items_df"]["count_of_items"]/df_dict["items_df"]["count_of_items"].sum())*100).round(2)
df_dict["items_df"]


,,item_name,count_of_items,perc_distinct_items
market_category,item_category,,,
Cinema,DVD,"['PHENOMENON (2012)', 'WINXCLUB. WITCH SCHOOL ...",4956,22.81
Music,locally produced CD,"['RAMAZZOTTI EROS Noi', 'PRODIGY The Fat Of Th...",2326,10.71
Cinema,Blu-Ray,"['PHENOMENON 2012 (BD)', 'AVATAR (BD+DVD)', 'A...",1770,8.15
PC Games,Digit,['Might and Magic: Heroes VI. Dance of Death [...,1098,5.05
Music,Vinyl,"['ARIA Asphalt Hero LP', 'Micah & Jumanji Bitc...",768,3.53
...,...,...,...,...
Android Games,Number,['Need for Speed Most Wanted [Android Digital]...,3,0.01
Books,Postcards,"['Postcard Lacarte CODE 5 (assorted)', 'Postca...",2,0.01
Accessories,PS2,['Pelican. Nerf Wireless Controller PS2 (assor...,2,0.01


In [18]:
df_dict["items_df_sales"] = df_dict["sales_df"].groupby(["market_category", "item_category"])["item_cnt_day"].sum().to_frame()

In [19]:
df_dict["items_df_sales"] = df_dict["items_df_sales"].sort_values(by="item_cnt_day", ascending=False)
df_dict["items_df_sales"]["perc_sales_items"] = ((df_dict["items_df_sales"]["item_cnt_day"]/df_dict["items_df_sales"]["item_cnt_day"].sum())*100)
df_dict["items_df_sales"]["accum_sales_cnt"] = df_dict["items_df_sales"]["perc_sales_items"].cumsum()
df_dict["items_df_sales"]

item_cnt_day  perc_sales_items  \
market_category item_category                                            
Cinema          DVD                         634171.0         17.765401   
PC Games        Standard Editions           456540.0         12.789321   
Music           locally produced CD         348591.0          9.765283   
Games           PS3                         254887.0          7.140298   
Cinema          Blu-Ray                     203284.0          5.694713   
...                                              ...               ...   
Books           Guides                           3.0          0.000084   
                Postcards                        3.0          0.000084   
Accessories     PS2                              2.0          0.000056   
Books           Educational Literature           1.0          0.000028   
Game consoles   PS2                              1.0          0.000028   

                                        accum_sales_cnt  
market_category item_category                            
Cinema          DVD                           17.765401  
PC Games        Standard Editions             30.554722  
Music           locally produced CD           40.320005  
Games           PS3                           47.460303  
Cinema          Blu-Ray                       53.155016  
...                                                 ...  
Books           Guides                        99.999804  
                Postcards                     99.999888  
Accessories     PS2                           99.999944  
Books           Educational Literature        99.999972  
Game consoles   PS2                          100.000000  

[77 rows x 3 columns]

In [20]:
df_dict["items_df_sales"]


item_cnt_day  perc_sales_items  \
market_category item_category                                            
Cinema          DVD                         634171.0         17.765401   
PC Games        Standard Editions           456540.0         12.789321   
Music           locally produced CD         348591.0          9.765283   
Games           PS3                         254887.0          7.140298   
Cinema          Blu-Ray                     203284.0          5.694713   
...                                              ...               ...   
Books           Guides                           3.0          0.000084   
                Postcards                        3.0          0.000084   
Accessories     PS2                              2.0          0.000056   
Books           Educational Literature           1.0          0.000028   
Game consoles   PS2                              1.0          0.000028   

                                        accum_sales_cnt  
market_category item_category                            
Cinema          DVD                           17.765401  
PC Games        Standard Editions             30.554722  
Music           locally produced CD           40.320005  
Games           PS3                           47.460303  
Cinema          Blu-Ray                       53.155016  
...                                                 ...  
Books           Guides                        99.999804  
                Postcards                     99.999888  
Accessories     PS2                           99.999944  
Books           Educational Literature        99.999972  
Game consoles   PS2                          100.000000  

[77 rows x 3 columns]

The 30.6 percent of sales is concentrated in sales of DVD and PC Games.


In [21]:
df_dict["item_sales_distr"] = df_dict["sales_df"].groupby(["market_category", "item_category"])["item_price"].describe()
df_dict["item_sales_distr"]

count         mean  \
market_category item_category                                         
Accessories     PS2                                2.0   215.500000   
                PS3                            18461.0  2261.705152   
                PS4                            25283.0  2105.683642   
                PSP                             2304.0   554.696081   
                PSVita                          7231.0   865.307153   
...                                                ...          ...   
Programs        Educational (Digital)           2346.0   270.454390   
                For home and office            42603.0  1859.867073   
                For home and office (Digital)   3746.0  1544.734347   
                MAC (Digital)                     56.0  3403.571429   
Service         Tickets                         1325.0  1246.064128   

                                                       std     min      25%  \
market_category item_category                                                 
Accessories     PS2                              95.459415  148.00   181.75   
                PS3                            1175.068988   93.00  1490.00   
                PS4                            1435.229478  122.59   599.00   
                PSP                             487.282737   22.00   299.00   
                PSVita                          757.659297   28.00   399.00   
...                                                    ...     ...      ...   
Programs        Educational (Digital)           405.439717   79.00   172.00   
                For home and office            2154.255419   91.00  1049.00   
                For home and office (Digital)  1711.924513   99.00   600.00   
                MAC (Digital)                  1858.811574  900.00  2780.00   
Service         Tickets                         208.086855  900.00  1100.00   

                                                  50%      75%       max  
market_category item_category                                             
Accessories     PS2                             215.5   249.25     283.0  
                PS3                            2390.0  2590.00    9990.0  
                PS4                            2090.0  3190.00    7490.0  
                PSP                             299.0   966.00    2590.0  
                PSVita                          699.0  1099.00    3490.0  
...                                               ...      ...       ...  
Programs        Educational (Digital)           188.0   228.00   14500.0  
                For home and office            1490.0  1990.00  307980.0  
                For home and office (Digital)  1180.0  1700.00   27900.0  
                MAC (Digital)                  2780.0  3648.00    8790.0  
Service         Tickets                        1300.0  1500.00    1700.0  

[77 rows x 8 columns]

In [22]:
df_dict["item_sales_distr_cnt"] = pd.merge(df_dict["item_sales_distr"], df_dict["items_df_sales"], on=["market_category", "item_category"],how="inner")

In [23]:
df_dict["item_sales_distr_cnt"].sort_values(by="item_cnt_day", ascending=False)

count         mean         std  \
market_category item_category                                               
Cinema          DVD                     564652.0   256.339150  128.605941   
PC Games        Standard Editions       351591.0   499.199532  346.719389   
Music           locally produced CD     339585.0   286.209370  113.765602   
Games           PS3                     208219.0  1512.044319  798.465332   
Cinema          Blu-Ray                 192674.0   434.289667  299.681484   
...                                          ...          ...         ...   
Books           Guides                       3.0   100.000000    0.000000   
                Postcards                    2.0    20.500000    4.949747   
Accessories     PS2                          2.0   215.500000   95.459415   
Books           Educational Literature       1.0   129.000000         NaN   
Game consoles   PS2                          1.0  1998.000000         NaN   

                                           min      25%     50%      75%  \
market_category item_category                                              
Cinema          DVD                       11.0   149.00   199.0   399.00   
PC Games        Standard Editions          0.5   249.00   399.0   599.00   
Music           locally produced CD       22.0   199.00   299.0   299.00   
Games           PS3                       -1.0   899.00  1299.0  1999.00   
Cinema          Blu-Ray                   22.0   199.00   398.0   549.00   
...                                        ...      ...     ...      ...   
Books           Guides                   100.0   100.00   100.0   100.00   
                Postcards                 17.0    18.75    20.5    22.25   
Accessories     PS2                      148.0   181.75   215.5   249.25   
Books           Educational Literature   129.0   129.00   129.0   129.00   
Game consoles   PS2                     1998.0  1998.00  1998.0  1998.00   

                                           max  item_cnt_day  \
market_category item_category                                  
Cinema          DVD                     1399.0      634171.0   
PC Games        Standard Editions       2399.0      456540.0   
Music           locally produced CD     3299.0      348591.0   
Games           PS3                     6999.0      254887.0   
Cinema          Blu-Ray                 3599.0      203284.0   
...                                        ...           ...   
Books           Guides                   100.0           3.0   
                Postcards                 24.0           3.0   
Accessories     PS2                      283.0           2.0   
Books           Educational Literature   129.0           1.0   
Game consoles   PS2                     1998.0           1.0   

                                        perc_sales_items  accum_sales_cnt  
market_category item_category                                              
Cinema          DVD                            17.765401        17.765401  
PC Games        Standard Editions              12.789321        30.554722  
Music           locally produced CD             9.765283        40.320005  
Games           PS3                             7.140298        47.460303  
Cinema          Blu-Ray                         5.694713        53.155016  
...                                                  ...              ...  
Books           Guides                          0.000084        99.999804  
                Postcards                       0.000084        99.999888  
Accessories     PS2                             0.000056        99.999944  
Books           Educational Literature          0.000028        99.999972  
Game consoles   PS2                             0.000028       100.000000  

[77 rows x 11 columns]

In [24]:
# Create a Dataframe for storing the items category sold by shop
df_dict["item_sales_shop"] = df_dict["sales_df"].groupby(["shop_name","market_category","item_category"])["item_cnt_day"].sum().to_frame()

df_dict["item_sales_shop"]["perc_sales_items"] = ((df_dict["item_sales_shop"]["item_cnt_day"]/df_dict["item_sales_shop"]["item_cnt_day"].sum())*100)
df_dict["item_sales_shop"] = df_dict["item_sales_shop"].sort_values(by="item_cnt_day", ascending=False)
df_dict["item_sales_shop"]["accum_sales_cnt"] = df_dict["item_sales_shop"]["perc_sales_items"].cumsum()

# Visualizing the first elements of the data
df_dict["item_sales_shop"].head(50)

item_cnt_day  \
shop_name                                     market_category item_category                            
Moscow shopping center "Semenovsky"           Cinema          DVD                            76069.0   
Moscow TRK "Atrium"                           Cinema          DVD                            45176.0   
Khimki Shopping Center "Mega"                 Cinema          DVD                            44904.0   
Yakutsk Ordzhonikidze, 56                     Cinema          DVD                            44589.0   
Moscow shopping center "MEGA Teply Stan" II   Cinema          DVD                            38588.0   
Digital warehouse 1C-Online                   PC Games        Digit                          35108.0   
Moscow shopping center "Semenovsky"           PC Games        Standard Editions              31908.0   
                                              Music           locally produced CD            31688.0   
Yakutsk shopping center "Central"             Cinema          DVD                            25663.0   
Moscow TRK "Atrium"                           Music           locally produced CD            22379.0   
St. Petersburg TC "Nevsky Center"             Cinema          DVD                            22179.0   
Chekhov SEC "Carnival"                        Cinema          DVD                            21463.0   
Moscow TRK "Atrium"                           PC Games        Standard Editions              21449.0   
Moscow shopping center "Semenovsky"           Gifts           Bags, Albums, Mouse pads       20182.0   
Moscow shopping center "MEGA Teply Stan" II   Music           locally produced CD            19081.0   
Khimki Shopping Center "Mega"                 Music           locally produced CD            19075.0   
Moscow TRK "Atrium"                           Cinema          Blu-Ray                        18792.0   
Moscow shopping center "MEGA Teply Stan" II   PC Games        Standard Editions              17024.0   
Moscow shopping center "MEGA Belaya Dacha II" Cinema          DVD                            16554.0   
Moscow shopping center "Semenovsky"           Cinema          Blu-Ray                        16431.0   
                                              Games           PS3                            16094.0   
Moscow shopping center "MEGA Belaya Dacha II" PC Games        Standard Editions              15792.0   
Khimki Shopping Center "Mega"                 PC Games        Standard Editions              15690.0   
Tyumen shopping center "Goodwin"              Cinema          DVD                            14703.0   
Tyumen shopping center "Green Coast"          Cinema          DVD                            14554.0   
Voronezh (Plekhanovskaya, 13)                 PC Games        Standard Editions              14406.0   
St. Petersburg TC "Nevsky Center"             PC Games        Standard Editions              14391.0   
Moscow TRK "Atrium"                           Gifts           Bags, Albums, Mouse pads       14343.0   
Kaluga SEC "XXI century"                      PC Games        Standard Editions              14069.0   
Voronezh (Plekhanovskaya, 13)                 Cinema          DVD                            13661.0   
Moscow TRK "Atrium"                           Games           PS3                            13596.0   
Moscow shopping center "MEGA Teply Stan" II   Gifts           Bags, Albums, Mouse pads       13458.0   
Moscow shopping center "Semenovsky"           PC Games        Extra Editions                 13226.0   
Moscow shopping center "MEGA Teply Stan" II   Games           PS3                            13001.0   
Kursk shopping center "Pushkinsky"            Cinema          DVD                            12938.0   
                                              PC Games        Standard Editions              12803.0   
Yakutsk Ordzhonikidze, 56                     PC Games        Standard Editions              12714.0   
St. Petersburg TC "Nevsky Center

In [25]:
df_dict["sales_shop"] = df_dict["sales_df"].groupby("shop_name")["item_cnt_day"].sum().to_frame().sort_values(by="item_cnt_day", ascending=False)
df_dict["sales_shop"]["perc_sales_cnt"] = ((df_dict["sales_shop"]["item_cnt_day"]/df_dict["sales_shop"]["item_cnt_day"].sum())*100)
df_dict["sales_shop"]["accum_sales_cnt"] = df_dict["sales_shop"]["perc_sales_cnt"].cumsum()
#df_dict["sales_shop"].reset_index().reset_index().set_index("shop_name")
df_dict["sales_shop"]["Ranking"] = [number + 1 for number in range(len(df_dict["sales_shop"]["accum_sales_cnt"]))]
#df_dict["sales_shop"]["Ranking_2"] = range(len(df_dict["sales_shop"]["accum_sales_cnt"])) 
df_dict["sales_shop"]["Ranking_%"] = ((df_dict["sales_shop"]["Ranking"]/df_dict["sales_shop"]["Ranking"].sum())*100).cumsum()
df_dict["sales_shop"]


,item_cnt_day,perc_sales_cnt,accum_sales_cnt,Ranking,Ranking_%
shop_name,,,,,
"Moscow shopping center ""Semenovsky""",310777.0,8.518625,8.518625,1,0.054645
"Moscow TRK ""Atrium""",241920.0,6.631204,15.149830,2,0.163934
"Khimki Shopping Center ""Mega""",185790.0,5.092640,20.242470,3,0.327869
"Moscow shopping center ""MEGA Teply Stan"" II",184557.0,5.058843,25.301312,4,0.546448
"St. Petersburg TC ""Nevsky Center""",144934.0,3.972747,29.274060,5,0.819672
"Yakutsk Ordzhonikidze, 56",141107.0,3.867846,33.141906,6,1.147541
"Moscow shopping center ""MEGA Belaya Dacha II""",136657.0,3.745869,36.887774,7,1.530055
"Voronezh (Plekhanovskaya, 13)",100489.0,2.754477,39.642252,8,1.967213
"Yakutsk shopping center ""Central""",81734.0,2.240389,41.882640,9,2.459016


In [26]:
df_dict["sales_df"].groupby("market_category")["item_cnt_day"].sum().to_frame().sort_values(by="item_cnt_day", ascending=False)

,item_cnt_day
market_category,
Cinema,916943.0
PC Games,689098.0
Games,646774.0
Gifts,584366.0
Music,411294.0
Accessories,85056.0
Books,76341.0
Programs,74954.0
Payment cards,46433.0


In [27]:
df_dict['sales_df']["year"] = df_dict['sales_df']["date"].dt.year
df_dict["tsa_market_cat"] = df_dict['sales_df'][["market_category", "item_cnt_day", "year"]]
df_dict["pivot_tsa_market_cat"] = pd.pivot_table(df_dict["tsa_market_cat"], values="item_cnt_day", index="market_category", columns=["year"], aggfunc=np.sum)
df_dict["pivot_tsa_market_cat"] = df_dict["pivot_tsa_market_cat"].fillna(0)
df_dict["pivot_tsa_market_cat"]["total"] = df_dict["pivot_tsa_market_cat"][2013] + df_dict["pivot_tsa_market_cat"][2014] + df_dict["pivot_tsa_market_cat"][2015]
df_dict["pivot_tsa_market_cat"] = df_dict["pivot_tsa_market_cat"].sort_values(by="total", ascending=False)
df_dict["pivot_tsa_market_cat"]

year,2013,2014,2015,total
market_category,,,,
Cinema,450848.0,306718.0,159377.0,916943.0
PC Games,328004.0,240449.0,120645.0,689098.0
Games,253819.0,244420.0,148535.0,646774.0
Gifts,200367.0,236018.0,147981.0,584366.0
Music,185993.0,144824.0,80477.0,411294.0
Accessories,32825.0,35030.0,17201.0,85056.0
Books,28221.0,23388.0,24732.0,76341.0
Programs,36216.0,24763.0,13975.0,74954.0
Payment cards,11230.0,19739.0,15464.0,46433.0


In [28]:
print(df_dict["pivot_tsa_market_cat"][2013].sum(),
      df_dict["pivot_tsa_market_cat"][2014].sum(),
      df_dict["pivot_tsa_market_cat"][2015].sum())

1562733.0 1320889.0 764584.0


In [29]:
df_dict["pivot_tsa_market_cat_%"] = pd.DataFrame()

df_dict["pivot_tsa_market_cat_%"][2013] = (df_dict["pivot_tsa_market_cat"][2013] / df_dict["pivot_tsa_market_cat"][2013].sum())
df_dict["pivot_tsa_market_cat_%"][2014] = (df_dict["pivot_tsa_market_cat"][2014] / df_dict["pivot_tsa_market_cat"][2014].sum())
df_dict["pivot_tsa_market_cat_%"][2015] = (df_dict["pivot_tsa_market_cat"][2015] / df_dict["pivot_tsa_market_cat"][2015].sum())
df_dict["pivot_tsa_market_cat_%"][["2013_pct_change", "2014_pct_change", "2015_pct_change"]] = df_dict["pivot_tsa_market_cat_%"].T.pct_change().T
df_dict["pivot_tsa_market_cat_%"]

,2013,2014,2015,2013_pct_change,2014_pct_change,2015_pct_change
market_category,,,,,,
Cinema,0.288500,0.232206,0.208449,NaN,-0.195127,-0.102308
PC Games,0.209891,0.182036,0.157792,NaN,-0.132714,-0.133183
Games,0.162420,0.185042,0.194269,NaN,0.139282,0.049864
Gifts,0.128216,0.178681,0.193544,NaN,0.393598,0.083183
Music,0.119018,0.109641,0.105256,NaN,-0.078782,-0.039998
Accessories,0.021005,0.026520,0.022497,NaN,0.262565,-0.151690
Books,0.018059,0.017706,0.032347,NaN,-0.019519,0.826868
Programs,0.023175,0.018747,0.018278,NaN,-0.191051,-0.025034
Payment cards,0.007186,0.014944,0.020225,NaN,1.079524,0.353436


In [30]:

df_dict["tsa_shops"] = df_dict['sales_df'][["shop_name", "item_cnt_day", "year"]]
df_dict["pivot_tsa_shops"] = pd.pivot_table(df_dict["tsa_shops"], values="item_cnt_day", index="shop_name", columns=["year"], aggfunc=np.sum)

df_dict["pivot_tsa_shops"] = df_dict["pivot_tsa_shops"].fillna(0)
df_dict["pivot_tsa_shops"]["total"] = df_dict["pivot_tsa_shops"][2013] + df_dict["pivot_tsa_shops"][2014] + df_dict["pivot_tsa_shops"][2015]
df_dict["pivot_tsa_shops"] = df_dict["pivot_tsa_shops"].sort_values(by=2015, ascending=False)

df_dict["pivot_tsa_shops"]["2013_%"] = (df_dict["pivot_tsa_shops"][2013] / df_dict["pivot_tsa_shops"][2013].sum())
df_dict["pivot_tsa_shops"]["2014_%"] = (df_dict["pivot_tsa_shops"][2014] / df_dict["pivot_tsa_shops"][2014].sum())
df_dict["pivot_tsa_shops"]["2015_%"] = (df_dict["pivot_tsa_shops"][2015] / df_dict["pivot_tsa_shops"][2015].sum())
df_dict["pivot_tsa_shops"][["2013_cumsum", "2014_cumsum", "2015_cumsum"]] = df_dict["pivot_tsa_shops"][["2013_%", "2014_%", "2015_%"]].cumsum()

df_dict["pivot_tsa_shops"] = df_dict["pivot_tsa_shops"].sort_values(by=2013, ascending=False)
df_dict["pivot_tsa_shops"]["Ranking_2013"] = [number + 1 for number in range(len(df_dict["pivot_tsa_shops"]["2013_%"]))]


df_dict["pivot_tsa_shops"] = df_dict["pivot_tsa_shops"].sort_values(by=2014, ascending=False)
df_dict["pivot_tsa_shops"]["Ranking_2014"] = [number + 1 for number in range(len(df_dict["pivot_tsa_shops"]["2014_%"]))]

df_dict["pivot_tsa_shops"] = df_dict["pivot_tsa_shops"].sort_values(by=2015, ascending=False)
df_dict["pivot_tsa_shops"]["Ranking_2015"] = [number + 1 for number in range(len(df_dict["pivot_tsa_shops"]["2015_%"]))]

df_dict["pivot_tsa_shops"]

year,2013,2014,2015,total,2013_%,2014_%,2015_%,2013_cumsum,2014_cumsum,2015_cumsum,Ranking_2013,Ranking_2014,Ranking_2015
shop_name,,,,,,,,,,,,,
"Moscow shopping center ""Semenovsky""",132487.0,110631.0,67659.0,310777.0,0.084779,0.083755,0.088491,0.084779,0.083755,0.088491,1,1,1
"Moscow TRK ""Atrium""",102572.0,82733.0,56615.0,241920.0,0.065636,0.062634,0.074047,0.150415,0.146389,0.162538,2,2,2
"St. Petersburg TC ""Nevsky Center""",53732.0,50134.0,41068.0,144934.0,0.034383,0.037955,0.053713,0.184799,0.184344,0.216251,7,7,3
"Moscow shopping center ""MEGA Teply Stan"" II",78657.0,65899.0,40001.0,184557.0,0.050333,0.049890,0.052317,0.235132,0.234234,0.268568,4,4,4
Digital warehouse 1C-Online,7433.0,24285.0,31670.0,63388.0,0.004756,0.018385,0.041421,0.239888,0.252619,0.309989,48,20,5
"Yakutsk Ordzhonikidze, 56",60163.0,52726.0,28218.0,141107.0,0.038499,0.039917,0.036906,0.278387,0.292536,0.346896,5,6,6
E-shop online,19201.0,26261.0,28016.0,73478.0,0.012287,0.019881,0.036642,0.290673,0.312418,0.383538,37,12,7
"Moscow shopping center ""MEGA Belaya Dacha II""",54426.0,55960.0,26271.0,136657.0,0.034827,0.042365,0.034360,0.325501,0.354783,0.417898,6,5,8
"Surgut SEC ""City Mall""",22291.0,25868.0,19478.0,67637.0,0.014264,0.019584,0.025475,0.339765,0.374367,0.443373,30,13,9


In [31]:
# Closed Shops.
list(df_dict["pivot_tsa_shops"].loc[df_dict["pivot_tsa_shops"][2015] == 0].index)

['Kazan shopping center "Behetle"',
 'Moscow shopping center "Silver House"',
 'Voronezh SEC City-Park "Grad"',
 '!Yakutsk shopping center "Central" franc',
 'Moscow TC "Budenovsky" (pavilion A2)',
 '!Yakutsk Ordzhonikidze, 56 francs']

In [32]:
df_dict["sales_market_shop_year"] = df_dict["sales_df"][["market_category", "shop_name", "year", "item_cnt_day"]]

df_dict["pivot_sales_market_shop_year"] = pd.pivot_table(df_dict["sales_market_shop_year"], values="item_cnt_day", index=["year","shop_name"], columns=["market_category"], aggfunc=np.sum)
df_dict["pivot_sales_market_shop_year"] = df_dict["pivot_sales_market_shop_year"].fillna(0)

df_dict["pivot_sales_market_shop_year"]





market_category                                Accessories  Android Games  \
year shop_name                                                              
2013 !Yakutsk Ordzhonikidze, 56 francs               403.0            0.0   
     !Yakutsk shopping center "Central" franc        202.0            0.0   
     Adygea shopping center "Mega"                   528.0            0.0   
     Balashikha TRC "October-Kinomir"                292.0            0.0   
     Chekhov SEC "Carnival"                          643.0            0.0   
...                                                    ...            ...   
2015 Yakutsk Ordzhonikidze, 56                       584.0            0.0   
     Yakutsk shopping center "Central"               462.0            0.0   
     Yaroslavl shopping center "Altair"              209.0            0.0   
     Zhukovsky str. Chkalova 39m?                     97.0            0.0   
     Zhukovsky str. Chkalova 39m²                     14.0            0.0   

market_category                                Batteries  Blank media (piece)  \
year shop_name                                                                  
2013 !Yakutsk Ordzhonikidze, 56 francs              86.0                146.0   
     !Yakutsk shopping center "Central" franc       64.0                169.0   
     Adygea shopping center "Mega"                  28.0                 70.0   
     Balashikha TRC "October-Kinomir"                0.0                  0.0   
     Chekhov SEC "Carnival"                        170.0                792.0   
...                                                  ...                  ...   
2015 Yakutsk Ordzhonikidze, 56                     187.0                  0.0   
     Yakutsk shopping center "Central"             105.0                  0.0   
     Yaroslavl shopping center "Altair"             93.0                  0.0   
     Zhukovsky str. Chkalova 39m?                   18.0                  0.0   
     Zhukovsky str. Chkalova 39m²                    0.0                  0.0   

market_category                                Books   Cinema  \
year shop_name                                                  
2013 !Yakutsk Ordzhonikidze, 56 francs         153.0   4678.0   
     !Yakutsk shopping center "Central" franc    5.0   2433.0   
     Adygea shopping center "Mega"             128.0   1414.0   
     Balashikha TRC "October-Kinomir"           27.0   2210.0   
     Chekhov SEC "Carnival"                    555.0  13105.0   
...                                              ...      ...   
2015 Yakutsk Ordzhonikidze, 56                 186.0  10637.0   
     Yakutsk shopping center "Central"         108.0   5944.0   
     Yaroslavl shopping center "Altair"         56.0   1832.0   
     Zhukovsky str. Chkalova 39m?               38.0    984.0   
     Zhukovsky str. Chkalova 39m²                1.0    109.0   

market_category                                Delivery of goods  \
year shop_name                                                     
2013 !Yakutsk Ordzhonikidze, 56 francs                       0.0   
     !Yakutsk shopping center "Central" franc                0.0   
     Adygea shopping center "Mega"                           0.0   
     Balashikha TRC "October-Kinomir"                        0.0   
     Chekhov SEC "Carnival"                                  0.0   
...                                                          ...   
2015 Yakutsk Ordzhonikidze, 56                               0.0   
     Yakutsk shopping center "Central"                       0.0   
     Yaroslavl shopping center "Altair"                      0.0   
     Zhukovsky str. Chkalova 39m?                            0.0   
     Zhukovsky str. Chkalova 39m²                            0.0   

market_category                                Game consoles   Games   Gifts  \
year shop_name                                                                 
2013 !Yakutsk Ordzhonikidze, 56 francs               

In [33]:
df_dict["sales_df"]["month"] = df_dict["sales_df"]["date"].dt.month

In [34]:
df_dict["tsa_markets_month"] = df_dict['sales_df'][["market_category", "item_cnt_day", "year", "month"]]
df_dict["pivot_tsa_markets_month"] = pd.pivot_table(df_dict["tsa_markets_month"], values="item_cnt_day", index=["year", "month"], columns="market_category", aggfunc=np.sum)
df_dict["pivot_tsa_markets_month"] = df_dict["pivot_tsa_markets_month"].fillna(0)
df_dict["pivot_tsa_markets_month"] = df_dict["pivot_tsa_markets_month"].T.sort_values(by=[(2015, 10)], ascending=False)

df_dict["tsa_markets_year"] = df_dict["pivot_tsa_markets_month"].stack().stack().reset_index()


df_dict["tsa_markets_year"].groupby(["year", "market_category"])[0].describe()

count         mean          std  \
year market_category                                                        
2013 Accessories                           12.0  2735.416667  1546.939939   
     Android Games                         12.0     0.000000     0.000000   
     Batteries                             12.0   499.333333   232.776652   
     Blank media (piece)                   12.0   822.250000   401.979675   
     Books                                 12.0  2351.750000   360.314541   
...                                         ...          ...          ...   
2015 Payment cards (Movie, Music, Games)   10.0     0.000000     0.000000   
     Programs                              10.0  1397.500000   351.812529   
     Pure media (spire)                    10.0     0.100000     0.316228   
     Service                               10.0  1340.400000  1534.269366   
     Tickets (Digital)                     10.0   611.200000  1594.037209   

                                             min      25%     50%      75%  \
year market_category                                                         
2013 Accessories                          1639.0  1961.25  2403.5  2592.75   
     Android Games                           0.0     0.00     0.0     0.00   
     Batteries                             168.0   276.00   525.0   675.25   
     Blank media (piece)                   178.0   463.50   955.0  1128.50   
     Books                                1886.0  2080.50  2273.5  2630.50   
...                                          ...      ...     ...      ...   
2015 Payment cards (Movie, Music, Games)     0.0     0.00     0.0     0.00   
     Programs                             1049.0  1121.00  1325.0  1591.75   
     Pure media (spire)                      0.0     0.00     0.0     0.00   
     Service                               521.0   695.25   875.5  1034.25   
     Tickets (Digital)                      -1.0     0.00    81.0   181.50   

                                             max  
year market_category                              
2013 Accessories                          7349.0  
     Android Games                           0.0  
     Batteries                             894.0  
     Blank media (piece)                  1374.0  
     Books                                2947.0  
...                                          ...  
2015 Payment cards (Movie, Music, Games)     0.0  
     Programs                             2105.0  
     Pure media (spire)                      1.0  
     Service                              5640.0  
     Tickets (Digital)                    5128.0  

[63 rows x 8 columns]

In [35]:
df_dict.keys()

dict_keys(['sales_df', 'sales_df_raw', 'items_cat_df', 'items_df', 'items_df_sales', 'item_sales_distr', 'item_sales_distr_cnt', 'item_sales_shop', 'sales_shop', 'tsa_market_cat', 'pivot_tsa_market_cat', 'pivot_tsa_market_cat_%', 'tsa_shops', 'pivot_tsa_shops', 'sales_market_shop_year', 'pivot_sales_market_shop_year', 'tsa_markets_month', 'pivot_tsa_markets_month', 'tsa_markets_year'])

In [36]:
cond = df_dict["tsa_market_cat"]["year"] == 2013
df_dict["tsa_market_cat"].loc[cond,:].pivot_table()

pd.pivot_table(df_dict["tsa_market_cat"].loc[cond,:], values="item_cnt_day", index="market_category", columns=["year"], aggfunc=np.sum).T
#
# df_dict["pivot_tsa_market_cat"] = df_dict["pivot_tsa_market_cat"].fillna(0)

#sns.set(rc={'figure.figsize':(20.7,16.27)})
#sns.heatmap(df_dict["pivot_tsa_market_cat"].T.loc[2013].corr(), annot=True)

ValueError: No group keys passed!

In [ ]:
df_dict["items_df_sales"].reset_index()[["market_category", "item_cnt_day"]]

,market_category,item_cnt_day
0,Cinema,634171.0
1,PC Games,456540.0
2,Music,348591.0
3,Games,254887.0
4,Cinema,203284.0
...,...,...
72,Books,3.0
73,Books,3.0
74,Accessories,2.0
75,Books,1.0


**Create a new feature call revenues.**



In [ ]:
import time

In [ ]:
def create_csv(dict_dataframe:dict, folder_path=str):
    """It generates mutiple csv files with the dataframes contained in a dictionary.
    
    This library uses the Path module from the pathlib library
    
    ## Parameters:
    * dict_dataframe:(dict) a dictionary which contains the name of the file, or dataset as a Key and the Dataframe as value.
    * folder_path: (str) the folder path in which it will create the csv files.
    ## Output:
    Multiple .csv files generated in the indicated path.
    """
    
    from pathlib import Path

    dataset_keys = list(dict_dataframe.keys())
    files = range(len(dict_dataframe.keys()))

    # Generate the path.
    for dataset in files:
        file_path = Path(folder_path + "\\" + dataset_keys[dataset] + ".csv")
        file_path.parent.mkdir(parents=True, exist_ok=True)  
        dict_dataframe[dataset_keys[dataset]].to_csv(file_path)
        print(f"The file for the: ", dataset_keys[dataset], " has been created in the following route: ", str(file_path),"\n")

    

    


In [ ]:
#start1 = time.time()
#folder_path ="C:\\Users\\Alejandro\\Desktop\\Alpha Project\\Code\\Supermarket_challenge\\Data_future_sales\\new_datasets"
#create_csv(dict_dataframe = df_dict, folder_path=folder_path)
#end1=time.time()

The file for the:  sales_df  has been created in the following route:  C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\sales_df.csv 

The file for the:  sales_df_raw  has been created in the following route:  C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\sales_df_raw.csv 

The file for the:  items_cat_df  has been created in the following route:  C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_cat_df.csv 

The file for the:  items_df  has been created in the following route:  C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_df.csv 

The file for the:  items_df_sales  has been created in the following route:  C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_df_sales.csv 

The file for the:  item_sales_distr  has been c

In [ ]:
import os

def create_csv_s(dict_dataframe: dict, folder_path: str):
    """It generates mutiple csv files with the dataframes contained in a dictionary.
    
    ## Parameters:
    * dict_dataframe:(dict) a dictionary which contains the name of the file, or dataset as a Key and the Dataframe as value.
    * folder_path: (str) the folder path in which it will create the csv files.
    ## Output:
    Multiple .csv files generated in the indicated path.
    """

    os.makedirs(folder_path, exist_ok=True)
    for dataset_name, dataset in dict_dataframe.items():
        file_path = os.path.join(folder_path, f"{dataset_name}.csv")
        dataset.to_csv(file_path)
        print(f"The file for the: {dataset_name} has been created in the following route: {file_path}\n")

In [ ]:
#start2 = time.time()
#data_loader_module.create_csv_s(dict_dataframe = df_dict, folder_path=folder_path)
#end2 = time.time()

The file for the: sales_df has been created in the following route: C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\sales_df.csv

The file for the: sales_df_raw has been created in the following route: C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\sales_df_raw.csv

The file for the: items_cat_df has been created in the following route: C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_cat_df.csv

The file for the: items_df has been created in the following route: C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_df.csv

The file for the: items_df_sales has been created in the following route: C:\Users\Alejandro\Desktop\Alpha Project\Code\Supermarket_challenge\Data_future_sales\new_datasets\items_df_sales.csv

The file for the: item_sales_distr has been created in the followin

In [ ]:
runtime1 = end1 - start1
runtime2 = end2 - start2

print(f"first: {runtime1}\nsecond: {runtime2}")

first: 170.14893984794617
second: 151.85465574264526


In [ ]:
import pickle


In [ ]:
folder_path = "C:\\Users\\Alejandro\\Desktop\\Alpha Project\\Code\\Supermarket_challenge\\Data_future_sales\\new_datasets"
create_pickle_file(object=df_dict, folder_path= folder_path, file_name="dict_dfs_eda" )

NameError: name 'create_pickle_file' is not defined

In [ ]:
import pickle
file = open("dict_EDA_dataframes", "wb")
pickle.dump(df_dict, file)
file.close()



In [ ]:
pd.to_pickle(df_dict)

TypeError: to_pickle() missing 1 required positional argument: 'filepath_or_buffer'